<h1>Tabla de contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Smart-UJ-UAQUE" data-toc-modified-id="Smart-UJ-UAQUE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>Smart-UJ UAQUE</strong></a></span><ul class="toc-item"><li><span><a href="#Documento-entendimiento-de-datos" data-toc-modified-id="Documento-entendimiento-de-datos-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Documento entendimiento de datos</a></span></li></ul></li><li><span><a href="#Importación-de-librerias" data-toc-modified-id="Importación-de-librerias-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Importación de librerias</a></span></li><li><span><a href="#Importación-de-datos" data-toc-modified-id="Importación-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importación de datos</a></span></li><li><span><a href="#Preparación-de-los-datos" data-toc-modified-id="Preparación-de-los-datos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preparación de los datos</a></span></li><li><span><a href="#Normalización-de-pesos" data-toc-modified-id="Normalización-de-pesos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Normalización de pesos</a></span></li><li><span><a href="#Agrupación-difusa" data-toc-modified-id="Agrupación-difusa-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Agrupación difusa</a></span></li></ul></div>

# **Smart-UJ UAQUE**
## Documento entendimiento de datos

Este Notebook implementa el algoritmo de perfil grupal para el trabajo de grado UAQUE. 
Los grupos son conformados a partir de los Deweys de los libros y se usará agrupación difusa para dar a cada
usuario un porcentaje de pertenencia a cada grupo. 

Para cumplir con el objetivo de negocio:

"Fomentar el descubrimiento de material bibliográfico que tiene altas probabilidades de interesar a un usuario a partir de sus gustos, hábitos y preferencias de lectura. "

Cada libro tendrá no solo un Dewey sino tres: Dewey unidad, Dewey decena y Dewey centena. Los cuales según el 
nivel de especificidad clasifica a un libro junto a otro de su misma categoría. Esto con el fin de que los 
usuario pertenezcan a grupos más generales donde pueden llegar conocer material bibliográfico nuevo que está 
relacionado con sus gustos y preferencias más es probable que no conozcan. 


**Integrantes:**
Johan Ortegón, 
Juan Angarita, 
Jorge Salgado, 
Daniel Tibaquira

**Directora de Proyecto de Grado:** 
Viky Arias

# Importación de librerias

In [112]:
import pandas as pd
import numpy as np
import time

* pandas: librería para importación, exportación, manejo y análisis de datos.
* numpy: ofrece operaciones matemáticas enfocadas en el manejo de arreglos y matrices.
* time: poder obtener el tiempo de inicio y finalización de las funciones.


# Importación de datos

* join: tabla de prestamos junto a la información de los ítems después de la limpieza y extracción de atributos adicionales. 
* feedback: tabla de esctructura similar a join, incluye columna adicional de calificación donde se muestra si el usuario le dio like o dislike a las recomendaciones mostradas.

In [113]:
#importamos la tabla de join 
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')

In [115]:
#importamos la tabla de feedback
feedback = pd.read_json('https://www.dropbox.com/s/0xczxjo927nsn0k/feedback_daniel.json?dl=1')

In [116]:
#Agregamos los feedback a la tabla de join
join = join.append(feedback)

In [117]:
#Eliminamos algunas columnas que no nos interesan para este notebook
join = join.drop(["Fecha","Dewey","Facultad","Temas","Union","TipoItem"], axis=1)
#Ejemplos y columnas del dataframe
display(join.head(3))
display(join.columns)

,RowID,IDItem,NumeroUbicacion,Ubicacion,Llave,Programa,IDUsuario,Year,Signatura,FechaCreacion,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena,calificacion
0,Row0,80000005327627,720.98 A71S,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.98,2013,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,720,720,700,NaN
1,Row1,80000001191496,720.9861 A71,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.9861,1993,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,720,720,700,NaN
2,Row2,80000004979759,540 CH15Q 2010,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,2011,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,540,540,500,NaN


Index(['RowID', 'IDItem', 'NumeroUbicacion', 'Ubicacion', 'Llave', 'Programa',
       'IDUsuario', 'Year', 'Signatura', 'FechaCreacion', 'Autor', 'Titulo',
       'AnioPublicacion', 'DeweyEspecifico', 'TemaDewey', 'DeweyUnidad',
       'DeweyDecena', 'DeweyCentena', 'calificacion'],
      dtype='object')

# Preparación de los datos
Crearemos la columna de pesos, así como el dataframe que identificará los gustos y preferencias de los usuarios. Estos dataframes se crearán teniendo en cuenta 3 columnas diferentes:
* DeweyUnidad 
* DeweyDecena
* DeweyCentena


**Creación de columna "Peso"**:
A partir del análisis con expertos en negocio, se ha entendido que es importante tomar en cuenta el cambio de gustos de los usuarios a través del tiempo y dar menos peso a prestamos que se realizaron en el pasado a prestamos más recientes. Para esto se creará una nueva columna denominada "Peso" la cual determinará el peso/importancia que tiene cada prestamo o feedback.
La disminución del peso será exponencial según la diferencia con el año actual y se calculará con la siguiente formula:

$Peso = \frac{1}{2^{(AñoActual-20xx)}}$


In [124]:
#Comenzamos a tomar el tiempo de ejecución total del algoritmo
start_time = time.time()

In [125]:
#Obtenemos el año actual
import datetime
now = datetime.datetime.now()
anio_actual = int(now.year)
print("Anio Actual: ", anio_actual)

Anio Actual:  2021


In [126]:
#Creamos la columna pesos para el dataframe a partir del año en que se realizó el prestamo
join["Peso"] = join.apply(lambda row: 1/2**(anio_actual-row.Year), axis=1 )
join[["Year","Peso"]]

,Year,Peso
0,2019,0.250
1,2019,0.250
2,2019,0.250
3,2019,0.250
4,2019,0.250
...,...,...
23,2019,0.250
24,2019,0.250
25,2018,0.125
26,2019,0.250


Todo prestamo realizado tendrá un peso positivo. Mientras que los datos provenientes del feedback pueden tener un peso positivo o negativo según las calificación del usuario(si dio like o dislike respectivamente).

In [127]:
join.loc[join.calificacion.isnull(), "calificacion"] = 1.0
join["Peso"] = join.apply(lambda row: (row.calificacion*row.Peso), axis=1 )

La siguiente función crea la tabla de preferencias y gustos de los usuarios a partir de los prestamos y la fecha de estos(Columna Peso).

Cada Fila representa un usuario, las columnas son los deweys y los datos es el peso acumulado que tiene un usuario frente a dicho dewey.

In [128]:
#crearTablaPesos: crea la tabla de pesos por usuario
#parámetros
    #columna = {deweyUnidad|deweyDecena|deweyCentena}
def crearTablaPesos(columna):
    #Tabla = pesos_usuarios
        #columnas = deweys
        #filas = usuarios
        #dato = peso que tiene el usuario en dicho dewey
        
    #creamos las columnas a partir de los deweys diferentes. 
    print("Creando tabla pesos ...Iniciando")
    agrupacion = join.groupby(["IDUsuario",columna])["Peso"].sum().reset_index(name="Peso")
    
    #Los pesos no deben ser menores a cero
    agrupacion.loc[agrupacion.Peso < 0.0, "Peso"] = 0
    
    #display(agrupacion.head(5))
    
    #cración del dataframe
    pesos_usuarios = pd.DataFrame()
    #Dataframe auxiliar para acelerar el proceso de concat
    aux = pd.DataFrame()
    #extremos los ids de todos los usuarios.
    ids = agrupacion["IDUsuario"].unique()
    print("Total de IDs de usuarios: ", len(ids))
    
    #Recorremos cada uno de los usuarios
    for usuario in ids:
        #obtenemos todos los prestamos del usuario
        prestamos = agrupacion.loc[agrupacion["IDUsuario"]==usuario]
        columnas = prestamos[columna].values
        pesos = prestamos["Peso"].values
        #Creamos la fila y la concatenamos al dataframe
        fila = pd.DataFrame(data = [pesos], columns = columnas)
        fila["IDUsuario"] = usuario
        aux = pd.concat([aux,fila])
        #cada 100 registros concatenamos al dataframe general
        if(aux.shape[0] == 100):
            pesos_usuarios = pd.concat([pesos_usuarios,aux])
            #Reiniciamos el dataframe
            aux = pd.DataFrame()
    pesos_usuarios = pd.concat([pesos_usuarios,aux])
    #Cambiamos los nil por 0
    pesos_usuarios = pesos_usuarios.fillna(0)
    pesos_usuarios.reset_index(drop=True, inplace=True)
    print("Creando tabla pesos ...Finalizando")
    return pesos_usuarios

Se crean 3 tablas de peso diferentes según la columna que se tuvo en cuenta:

* DeweyUnidad
* DeweyDecena
* DeweyCentena

In [129]:
pesos_usuarios_unidad=crearTablaPesos("DeweyUnidad")
display(pesos_usuarios_unidad.head(5))
print("Forma de la tabla: ", pesos_usuarios_unidad.shape)

Creando tabla pesos ...Iniciando
Total de IDs de usuarios:  24407
Creando tabla pesos ...Finalizando


,658,IDUsuario,378,657,1,227,347,350,808,812,...,964,367,383,494,413,496,799,855,876,97
0,0.5,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.375,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,000a0630c32a437052696f6921e3181f155291d0,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,000c040064aedd76057c5309064a3823db970644,0.000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,000f23e62e6d4995ede7090c6447ad6736edc4de,0.000,0.000,0.0,0.5,0.5,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 756)


In [130]:
pesos_usuarios_decena = crearTablaPesos("DeweyDecena")
display(pesos_usuarios_decena.head(5))
print("Forma de la tabla: ", pesos_usuarios_decena.shape)

Creando tabla pesos ...Iniciando
Total de IDs de usuarios:  24407
Creando tabla pesos ...Finalizando


,650,IDUsuario,370,0,220,340,350,800,810,20,...,560,640,470,310,10,480,90,990,50,30
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,0.000,0.0,0.5,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 100)


In [131]:
pesos_usuarios_centena = crearTablaPesos("DeweyCentena")
display(pesos_usuarios_centena.head(5))
print("Forma de la tabla: ", pesos_usuarios_centena.shape)

Creando tabla pesos ...Iniciando
Total de IDs de usuarios:  24407
Creando tabla pesos ...Finalizando


,600,IDUsuario,300,0,200,800,700,100,400,500,900,-900
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,1.000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 12)


Exportamos e Importamos las tablas para agilizar el proceso

In [104]:
pesos_usuarios_unidad.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_unidad.json')
pesos_usuarios_decena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_decena.json')
pesos_usuarios_centena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_centena.json')

In [11]:
pesos_usuarios_unidad = pd.read_json('https://www.dropbox.com/s/aitygqwn9q47rlg/pesos_usuario_x_dewey_unidad.json?dl=1')

In [12]:
pesos_usuarios_decena = pd.read_json('https://www.dropbox.com/s/vr6ehn8xjhojuba/pesos_usuario_x_dewey_decena.json?dl=1')

In [13]:
pesos_usuarios_centena = pd.read_json('https://www.dropbox.com/s/2vnntgjnqpijkgg/pesos_usuario_x_dewey_centena.json?dl=1')

# Normalización de pesos
En busca de que dos usuarios con mismos gustos, pero diferente cantidad de préstamos sean equivalentes se decidió por normalizar cada una de las filas. Así un usuario con 3 préstamos en el Dewey 600 y otro con un único préstamo en el Dewey 600 se vuelven equivalente al tener gustos únicamente por este Dewey.

In [132]:
#función: normalizar las filas por valores entre cero y uno.
    #Parámetros: pesos_usuario, representa la matriz dispersa que se va a normalizar.
    #{pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
def normalizar_pesos(pesos_usuario):
    print("Normalizando pesos...Iniciando")
    usuarios = pesos_usuario['IDUsuario']
    #eliminamos momentaneamente la columna de IDUsuario para poder normalizar el resto de la tabla
    pesos_usuario = pesos_usuario.apply(pd.to_numeric, errors='coerce').drop(["IDUsuario"],axis=1)
    #display(pesos_usuario)
    #Normalizamos por fila
    sumatoria = pesos_usuario.max(axis=1)
    pesos_norm = pesos_usuario.div(pesos_usuario.sum(axis=1), axis=0)
    pesos_norm_id = pesos_norm.copy()
    # Agregamos nuevamente el IDUsuario
    if len(pesos_norm_id) == len(join.IDUsuario.unique()):
        pesos_norm_id['IDUsuario'] = usuarios
    print("Normalizando pesos...Acabado")
    return pesos_norm_id

In [133]:
pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios_unidad)
display(pesos_norm_id_unidad.shape)
pesos_norm_id_unidad.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 756)

,658,378,657,1,227,347,350,808,812,813,...,367,383,494,413,496,799,855,876,97,IDUsuario
0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.0,0.25,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.0,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [134]:
pesos_norm_id_decena = normalizar_pesos(pesos_usuarios_decena)
display(pesos_norm_id_decena.shape)
pesos_norm_id_decena.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 100)

,650,370,0,220,340,350,800,810,20,330,...,640,470,310,10,480,90,990,50,30,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [135]:
pesos_norm_id_centena = normalizar_pesos(pesos_usuarios_centena)
display(pesos_norm_id_centena.shape)
pesos_norm_id_centena.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 12)

,600,300,0,200,800,700,100,400,500,900,-900,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


Exportamos la tabla de pesos normalizados para el dewey por unidad con el fin
de poder mostrar los  gustos y preferencias de los usuarios en el dashboard.

In [96]:
pesos_norm_id_unidad.to_json(r'C:\Users\user\Downloads\pesos_norm_id_unidad.json')


# Agrupación difusa
Para la agrupación cada usuario podrá pertenecer a uno o más Dewey. Cada usuario tendrá un grado de pertenencia a un Dewey en específico dependiendo de sus gustos y preferencias resumidos en las matrices dispersas. 
La función se debe recorrer mínimo 3 veces con diferentes matrices de dispersión:
* pesos_usuarios_unidad
* pesos_usuarios_decena
* pesos_usuarios_centena

In [136]:
#Elimina los valores diferentes de cero para una fila especifica
#Sirve para visualizar únicamente los deweys donde el usuario tiene prestamos. 
def eliminar_cero(id_usuario, df_pesos):
    m1 = (df_pesos['IDUsuario'] == id_usuario)
    m2 = (df_pesos[m1] != 0).all()
    return df_pesos.loc[m1,m2]

In [95]:
#usuario de prueba
eliminar_cero("6466dbb15c41fdacb59eb1179817958de2c57191", pesos_norm_id_unidad)

,823,621,833,839,834,IDUsuario
9664,0.061224,0.020408,0.591837,0.244898,0.081633,6466dbb15c41fdacb59eb1179817958de2c57191


In [137]:
#Esta función asocia una grado de pertenencia a cada usuario
#Parámetros:
    #pesos_usuarios: matrix dispersa de pesos {pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
    #nombre_archivo: nombre del archivo a exportar con la tabla de df_pertenencia
#se crea la tabla df_pertenencia: describe la pertenecia de cada usuario a un cluster(dewey)
def clustering(pesos_usuarios):
    print("Comenzando clustering...")
    #normalizamos los pesos
    pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios)
    #ponemos el usuario a ambos dataframes
    pesos_usuarios["IDUsuario"] = pesos_norm_id_unidad["IDUsuario"]
    #Creamos la estructura del dataframe
    df_pertenencia = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia','Peso'])
    #iteramos por cada fila(usuario) del dataframe
    for index, row in pesos_norm_id_unidad.iterrows():
        #Extraemos los usuarios
        usuario = row["IDUsuario"]
        #Extraemos los pesos diferentes de cero normalizados y sin normalizar
        usuario_limpio_norm = eliminar_cero(row["IDUsuario"], pesos_norm_id_unidad)
        usuario_limpio_orig = eliminar_cero(row["IDUsuario"], pesos_usuarios)
        #obtenemos los pesos y pertenencias de cada usuario a cada grupo
        pertenecias = usuario_limpio_norm.drop(["IDUsuario"], axis=1).values
        pesos = usuario_limpio_orig.drop(["IDUsuario"], axis=1).values
        #los clusters son las columnas de estos dataframes
        clusters = usuario_limpio_norm.drop(["IDUsuario"], axis=1).columns
        #creamos un array que tenga las veces necesarias al usuario
        tamanio = len(clusters)
        usuarios = np.repeat([usuario], tamanio)
        #agregamos la fila al dataframe
        fila = pd.DataFrame(data = {'IDUsuario': usuarios, 'Cluster': clusters
                                   , 'Pertenencia': pertenecias[0], 'Peso': pesos[0]})
        df_pertenencia = df_pertenencia.append(fila)
    print("Finalizando clustering...")
    return df_pertenencia

In [138]:
pesos_clustering_unidad = clustering(pesos_usuarios_unidad)
display(pesos_clustering_unidad.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.25,0.125
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.75,0.375


In [139]:
pesos_clustering_decena = clustering(pesos_usuarios_decena)
display(pesos_clustering_decena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.25,0.125


In [140]:
pesos_clustering_centena = clustering(pesos_usuarios_centena)
display(pesos_clustering_centena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.25,0.125


In [141]:
pesos_clustering_unidad.reset_index(drop=True, inplace=True)
pesos_clustering_decena.reset_index(drop=True, inplace=True)
pesos_clustering_centena.reset_index(drop=True, inplace=True)

In [142]:
#Tiempo total de ejecución del algoritmo(excluyendo importación y exportación de datos)
final_time = time.time()
print("--- %s seconds ---" % (final_time - start_time))

--- 985.8973426818848 seconds ---


# Exportación de datos finales

Exportamos e importamos los datos para agilizar el proceso

In [1]:
pesos_clustering_unidad.to_json(r'C:\Users\user\Downloads\pesos_clustering_unidad.json')
pesos_clustering_decena.to_json(r'C:\Users\user\Downloads\pesos_clustering_decena.json')
pesos_clustering_centena.to_json(r'C:\Users\user\Downloads\pesos_clustering_centena.json')

In [25]:
pesos_clustering_unidad = pd.read_json('https://www.dropbox.com/s/6j30n8y3fn8358l/pesos_clustering_unidad.json?dl=1')
pesos_clustering_decena = pd.read_json('https://www.dropbox.com/s/6m7vbpfq8b8qz4s/pesos_clustering_decena.json?dl=1')
pesos_clustering_centena = pd.read_json('https://www.dropbox.com/s/3rjqco5swu55cna/pesos_clustering_centena.json?dl=1')